In [1]:
import pandas as pd 
import numpy as np

import sbmlcore

/Users/viktoriabrunner/Documents/Studium/PhD/DPhil/predict_FQ_resistance/predict_FQ_resistance/.venv_fq/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
data = pd.read_csv('data/ML_DATA_LEV.csv')
data.dropna(inplace= True)

#remove line below to train on samples rather than mutations
data.drop_duplicates(subset=['GENE_MUTATION'], keep='first', inplace=True)
data.GENE_MUTATION.value_counts()

GENE_MUTATION
gyrA_A90V     1
gyrB_K668R    1
gyrA_T164M    1
gyrA_Y379C    1
gyrB_G589V    1
             ..
gyrA_D488A    1
gyrA_P119L    1
gyrA_G477E    1
gyrB_A643D    1
gyrA_D94V     1
Name: count, Length: 163, dtype: int64

In [12]:
features = ['d_volume', 'd_hydropathy_KD', 'd_hydropathy_WW',
       'd_MW', 'd_Pi', 'secondary_structure', 'phi', 'psi', 'residue_sasa', 
       'n_hbond_acceptors', 'n_hbond_donors', 'SASA', 'snap2_score', 'dist_MGB', 'dist_MGE', 'dist_drug', 'temp_factor', 'depth', 'deep_ddG'] # 'B', 'C', 'E', 'G', 'H', 'T',

features = data[features]
features['secondary_structure_codes'] = pd.Categorical(features.secondary_structure, categories=features.secondary_structure.unique()).codes
features.drop('secondary_structure', inplace = True, axis = 1)

/var/folders/nq/gpbdk7y11kvd1d9nk9s77b4c0000gn/T/ipykernel_38418/812203343.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['secondary_structure_codes'] = pd.Categorical(features.secondary_structure, categories=features.secondary_structure.unique()).codes
/var/folders/nq/gpbdk7y11kvd1d9nk9s77b4c0000gn/T/ipykernel_38418/812203343.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.drop('secondary_structure', inplace = True, axis = 1)


In [13]:
features

,d_volume,d_hydropathy_KD,d_hydropathy_WW,d_MW,d_Pi,phi,psi,residue_sasa,n_hbond_acceptors,n_hbond_donors,SASA,snap2_score,dist_MGB,dist_MGE,dist_drug,temp_factor,depth,deep_ddG,secondary_structure_codes
0,51.4,2.4,-0.86,28.0,-0.04,-61.83,-59.76,47.0,2.0,0.0,55.555816,-64.0,14.254302,5.504266,10.310827,40.439999,1.665078,0.089,0
1,82.5,2.2,-2.18,48.1,2.89,-58.04,-41.61,97.4,1.0,1.0,88.155089,81.0,18.193078,7.778979,14.095746,46.660000,1.733342,-0.376,0
2,3.0,0.0,-1.98,-1.0,2.64,-58.04,-41.61,97.4,1.0,1.0,88.155089,56.0,18.193078,7.778979,14.095746,46.660000,1.733342,-0.347,0
3,-3.0,0.0,1.98,1.0,-2.64,-66.66,120.50,43.0,2.0,2.0,40.444768,53.0,21.565351,18.549803,13.333886,62.860001,2.030527,-0.309,1
4,-25.1,2.7,-0.10,-27.0,0.27,-78.75,123.63,5.4,2.0,0.0,3.978588,-22.0,38.286533,31.644022,37.267766,50.119999,3.114263,-1.229,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1623,-50.6,-5.2,0.92,-12.1,-0.42,-130.11,149.27,24.8,0.0,0.0,25.392934,-40.0,27.224609,38.799515,38.672577,96.040001,1.915598,-1.102,5
1628,-64.9,7.0,-0.78,-53.0,-5.69,-58.30,-53.31,99.4,1.0,1.0,108.315927,-17.0,43.646112,41.821476,47.286371,59.320000,1.914005,-0.669,0
1656,-113.3,4.1,0.14,-99.1,-4.79,-79.33,120.91,123.9,1.0,0.0,117.789422,25.0,39.778686,31.598502,35.352796,61.540001,1.769443,-1.407,3
1706,-54.5,0.4,-1.38,-14.1,-4.33,-96.55,-13.55,131.0,0.0,1.0,129.497644,-54.0,29.899138,38.862500,33.061004,91.370003,1.615220,-0.314,4


In [ ]:
X = np.column_stack([np.ones(len(data), dtype=np.float32), features['d_volume'].values])
y = data.CLEAN_BINARY_PHENOTYPE

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score

# Model fitting
lr = LinearRegression()
model = lr.fit(X,y)
print(f'R Squared: {model.score(X,y)}')
# Model coefficients
print(f'intercept: {model.intercept_} | coef: {model.coef_}')

# Model prediction
skl_preds = model.predict(X)
model.predict(new_x.reshape(1,-1)) # For new X value

# Model Evaluation
explained_variance = explained_variance_score(y,skl_preds) # explained variance = r2
r_squared = r2_score(y,skl_preds) # r square
mae = mean_absolute_error(y,skl_preds) # Mean absolute Percentag error
mse = mean_squared_error(y,skl_preds) # Mean squared error
print(f'explained Var.: {explained_variance}\n\nR Square: {r_squared}\n\nMean Abs Error:{mae}\n\nMean Squared Error:{mse}')
